## Examining the GTDB output

This notebook does the following:

- Checking if clusters contian multiple species, genus, or families
- generate dataframes containing all the resulting annotations and on for only Lactobacillaceae


In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn 
import numpy as np
import seaborn as sns

#### Open GTDBtk results (bac120andAr122.summary.tsv)

In [2]:
f = open("../data56_related_files/gtdbtk.bac_ar_summary_relevant_columns.tsv", "r")
files = f.read()

#### Create list and dataframe 

In [3]:
fileList = files.split("\n")

In [4]:
df = pd.DataFrame(fileList)

In [5]:
df

,0
0,user_genome\tclassification\tclosest_placement...
1,S10C1032\td__Bacteria;p__Bacteroidota;c__Bacte...
2,S10C1039\td__Bacteria;p__Firmicutes;c__Bacilli...
3,S10C1113\td__Bacteria;p__Firmicutes;c__Bacilli...
4,S10C119\td__Bacteria;p__Firmicutes;c__Bacilli;...
...,...
2380,user_genome\tclassification\tclosest_placement...
2381,S23C1277\td__Archaea;p__Methanobacteriota;c__M...
2382,S34C1277\td__Archaea;p__Methanobacteriota;c__M...
2383,S53C3468\td__Archaea;p__Methanobacteriota;c__M...


#### Cleaning df and splitting string to columns

In [6]:
df.columns = ['temp']
# user_genome	classification	fastani_reference	fastani_reference_radius	fastani_taxonomy	
# fastani_ani	fastani_af	closest_placement_reference	closest_placement_radius	closest_placement_taxonomy	
# closest_placement_ani	closest_placement_af	pplacer_taxonomy	classification_method	note	
# other_related_references(genome_id,species_name,radius,ANI,AF)	msa_percent	translation_table	red_value
# warnings

df[['user_genome','classification', 'closest_placement_taxonomy',
    'msa_percent']] = df.temp.str.split("\t",expand=True)


In [7]:
df.head(3)

,temp,user_genome,classification,closest_placement_taxonomy,msa_percent
0,user_genome\tclassification\tclosest_placement...,user_genome,classification,closest_placement_taxonomy,msa_percent
1,S10C1032\td__Bacteria;p__Bacteroidota;c__Bacte...,S10C1032,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,33.75
2,S10C1039\td__Bacteria;p__Firmicutes;c__Bacilli...,S10C1039,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,94.58


In [8]:
# drop first row because we get double headers 
df = df.drop(0)

# drop the last row since it is empty
last_row = len(df)
df = df.drop(df.index[last_row-1]) 

In [9]:
# drop first coulmn that is the original combined string
df = df.iloc[: , 1:]

In [10]:
# Because archea and bacteria were concatinated the header line is present further down where the archea file starts. 
df.drop(index=df[df['user_genome'] == 'user_genome'].index, inplace=True)

In [11]:
df.head(5)

,user_genome,classification,closest_placement_taxonomy,msa_percent
1,S10C1032,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,33.75
2,S10C1039,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,94.58
3,S10C1113,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,96.07
4,S10C119,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,67.12
5,S10C1295,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,44.93


In [12]:
#splitting classification to columns based on tax-rank
df[['Domain','Phylum', 'Class', 'Order', 'Family', 
    'Genus', 'Species']] = df.classification.str.split(";",expand=True)
#splitting user_genome ID to sample and cluster
df[['sample','cluster']] = df.user_genome.str.split("C",expand=True)

In [13]:
df.head(5)

,user_genome,classification,closest_placement_taxonomy,msa_percent,Domain,Phylum,Class,Order,Family,Genus,Species,sample,cluster
1,S10C1032,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,33.75,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Phocaeicola,s__Phocaeicola dorei,S10,1032
2,S10C1039,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,94.58,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Limosilactobacillus,s__Limosilactobacillus ingluviei,S10,1039
3,S10C1113,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,96.07,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Limosilactobacillus,s__Limosilactobacillus oris,S10,1113
4,S10C119,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,67.12,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Weissella,s__Weissella thailandensis,S10,119
5,S10C1295,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,44.93,d__Bacteria,p__Campylobacterota,c__Campylobacteria,o__Campylobacterales,f__Helicobacteraceae,g__Helicobacter_D,s__Helicobacter_D pullorum,S10,1295


In [14]:
df.cluster = df.cluster.astype(int)

In [15]:
df.to_csv('../data56_related_files/GTDB_All.csv', index = False)

## Sanity Check
Check if the bins in the same cluster has recieved the same taxonomical assignation

In [17]:
def CheckingIdenticalTaxInCluster_all(df):
    '''
    identify clusters containing different species
    1. group by cluster
    2. identify/log unique species, where species are more than one - exclude s__
    3. if species is undefined - s__ , identify unique genus for s__, where genus are more than one
    4. if genus is undefined, identify unique family, where family are more than one'''

    df.sort_values(by=['cluster'], inplace=True)
    f = open("../data56_related_files/log_cluster_irregularities.csv", "a")
    
    grouped = df.groupby('cluster')

    for index, c in grouped:
        cluster = c['cluster'].unique()
        
        uniqueGenus = []
        uniqueFamily = []
        
        # value in cluster is unique
        uniqueSpecies = c['Species'].unique()
    
        if 's__' in uniqueSpecies:
            # check genus
            uniqueGenus = c['Genus'].unique()
            if 'g__' in uniqueGenus:
                # check family
                uniqueFamily = c['Family'].unique()


        
        if (len(uniqueSpecies) > 1 or len(uniqueGenus) > 1 or len(uniqueFamily) > 1):
            
            if (len(np.delete(uniqueSpecies, np.where(uniqueSpecies == "s__"))) < 2 and 
                len(np.delete(uniqueGenus, np.where(uniqueGenus == "g__"))) < 2 and len(uniqueFamily) < 2):
                continue
                
            # there is more than one - now log it
            f.write(f"Species: {uniqueSpecies}, Genus: {uniqueGenus}, Family: {uniqueFamily}, Cluster: {cluster} \n")
        
    f.close()
    
CheckingIdenticalTaxInCluster_all(df)

In [18]:
# !rm data56_related_files/log_cluster_irregularities.csv

In [19]:
df.loc[df['cluster'] == 1]

,user_genome,classification,closest_placement_taxonomy,msa_percent,Domain,Phylum,Class,Order,Family,Genus,Species,sample,cluster
727,S27C1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,N/A,11.12,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Coprobacteraceae,g__,s__,S27,1
1868,S50C1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,15.39,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Barnesiellaceae,g__Barnesiella,s__,S50,1
659,S26C1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,N/A,19.0,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Coprobacteraceae,g__,s__,S26,1
1543,S45C1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,N/A,10.56,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Coprobacteraceae,g__,s__,S45,1


Number of undefined at each rank:

In [20]:
df[df["Species"] == 's__'].count()['user_genome']

1093

In [21]:
df[df["Genus"] == 'g__'].count()['user_genome']

108

In [22]:
df[df["Family"] == 'f__'].count()['user_genome']

0

In [23]:
df.nunique()

user_genome                   2382
classification                 467
closest_placement_taxonomy     397
msa_percent                   1786
Domain                           2
Phylum                          17
Class                           21
Order                           52
Family                         100
Genus                          246
Species                        269
sample                          56
cluster                       1067
dtype: int64

### Overview
Out of 2382 bins/rows 

* 1093 bins with species undefined 
* 108 bins with genus undefined 
* 0 bins with families undefined

Out of 1067 clusters 
* 2 times did species differ in two clusters - s__Corynebacterium pollutisoli/s__Corynebacterium xerosis, and s__Escherichia flexneri/s__Escherichia coli_D in cluster 456 and 672
* 4 times did genus differ within a cluster 
* 6 times did families differ within a cluster - in all cases the confusion was between f__Coprobacteraceae and f__Barnesiellaceae. 

see more in the log_clusters_irregularities file. 

### Creating GTDB dataframe for Lactobacillaceae bins

In [32]:
Lactobacillaceae_df =df[df["Family"] == 'f__Lactobacillaceae']

In [33]:
Lactobacillaceae_df = pd.DataFrame(Lactobacillaceae_df)

In [34]:
Lactobacillaceae_df.to_csv('../data56_related_files/GTDB_Lactobacillaceae.csv', index = False)

In [36]:
Lactobacillaceae_df.head(5)

,user_genome,classification,closest_placement_taxonomy,msa_percent,Domain,Phylum,Class,Order,Family,Genus,Species,sample,cluster
1545,S45C105,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,40.0,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Limosilactobacillus,s__,S45,105
643,S25C105,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,73.26,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Limosilactobacillus,s__,S25,105
1782,S49C105,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,53.94,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Limosilactobacillus,s__,S49,105
1322,S39C105,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,65.52,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Limosilactobacillus,s__,S39,105
2250,S8C119,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,77.29,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Weissella,s__Weissella thailandensis,S8,119


In [32]:
Bacillus_df =df[df["Class"] == 'c__Bacilli']

In [33]:
Bacillus_df

,user_genome,classification,closest_placement_taxonomy,msa_percent,Domain,Phylum,Class,Order,Family,Genus,Species,sample,cluster
2322,S8C7,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,17.63,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Aerococcaceae,g__Aerococcus,s__Aerococcus urinaeequi,S8,7
209,S14C7,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,17.17,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Aerococcaceae,g__Aerococcus,s__Aerococcus urinaeequi,S14,7
1379,S3C7,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,80.5,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Aerococcaceae,g__Aerococcus,s__Aerococcus urinaeequi,S3,7
250,S15C7,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,69.09,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Aerococcaceae,g__Aerococcus,s__Aerococcus urinaeequi,S15,7
373,S19C39,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,91.9,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Enterococcaceae,g__Enterococcus_B,s__Enterococcus_B faecium,S19,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,S11C10001,d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...,d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...,10.86,d__Bacteria,p__Firmicutes,c__Bacilli,o__RF39,f__UBA660,g__CAG-877,s__,S11,10001
2098,S55C13176,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,17.65,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Ligilactobacillus,s__Ligilactobacillus salivarius,S55,13176
2173,S6C25284,d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...,d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...,16.97,d__Bacteria,p__Firmicutes,c__Bacilli,o__RF39,f__UBA660,g__CAG-1193,s__,S6,25284
231,S15C25531,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,10.8,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Ligilactobacillus,s__Ligilactobacillus salivarius,S15,25531


In [34]:
Bacillus_df['Genus'].unique()

array(['g__Aerococcus', 'g__Enterococcus_B', 'g__Holdemania',
       'g__Streptococcus', 'g__Nosocomiicoccus', 'g__Limosilactobacillus',
       'g__Weissella', 'g__HGM10873', 'g__Merdibacter',
       'g__Jeotgalicoccus', 'g__Staphylococcus',
       'g__Pseudogracilibacillus', 'g__Lactobacillus', 'g__Turicibacter',
       'g__Atopostipes', 'g__Staphylococcus_A', 'g__Massilicoli',
       'g__Erysipelatoclostridium', 'g__Ligilactobacillus', 'g__UMGS2068',
       'g__Jeotgalicoccus_A', 'g__CAG-988', 'g__UMGS268',
       'g__Facklamia_A', 'g__Enterococcus', 'g__CAG-451',
       'g__Faecalicoccus', 'g__CAG-594', 'g__CAG-822', 'g__Globicatella',
       'g__Enterococcus_E', 'g__Levilactobacillus', 'g__Paenibacillus',
       'g__CAG-877', 'g__UBA733', 'g__CAG-776', 'g__Companilactobacillus',
       'g__', 'g__Ureaplasma', 'g__Amedibacterium', 'g__Kurthia',
       'g__CAG-1193', 'g__UBA4855', 'g__Aliicoccus', 'g__CAG-460',
       'g__Atopococcus', 'g__Faecalitalea'], dtype=object)